In [1]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import requests
import tqdm
from time import sleep
import time
from PIL import Image
from io import BytesIO
from adjustText import adjust_text

from nba_api.stats.static.players import find_players_by_full_name
from nba_api.stats.static.teams import find_teams_by_full_name, get_teams, find_team_name_by_id
from nba_api.stats.endpoints.playbyplayv2 import PlayByPlayV2
from nba_api.stats.endpoints.leaguegamefinder import LeagueGameFinder
from nba_api.stats.endpoints.shotchartdetail import ShotChartDetail


from nba_api.stats.endpoints.leaguedashplayerstats import LeagueDashPlayerStats
from collections import Counter

import json

In [3]:
real_shots = sorted(list(set(['Alley Oop Dunk Shot',
                            'Alley Oop Layup shot',
                            'Cutting Dunk Shot',
                            'Cutting Finger Roll Layup Shot',
                            'Cutting Layup Shot',
                            'Dunk Shot',
                            'Jump Bank Shot',
                            'Jump Shot',
                            'Layup Shot',
                            'Hook Shot',
                            'Reverse Dunk Shot',
                            'Reverse Layup Shot',
                            'Running Alley Oop Dunk Shot',
                            'Running Alley Oop Layup Shot',
                            'Running Dunk Shot',
                            'Running Finger Roll Layup Shot',
                            'Running Jump Shot',
                            'Running Layup Shot',
                            'Running Pull-Up Jump Shot',
                            'Running Reverse Layup Shot',
                            "Hook Bank Shot",
                            "Finger Roll Layup Shot",
                            "Running Reverse Dunk Shot",
                            "Slam Dunk Shot",
                            "Jump Hook Shot",
                            "Running Hook Shot",
                            "Follow Up Dunk Shot",
                            "Finger Roll Shot",
                            "Running Finger Roll Shot",
                            "Running Tip Shot",
                            "Jump Shot",
                            "Layup Shot",
                            "Dunk Shot",
                            "Hook Shot",
                            'Running Slam Dunk Shot',
                            'Running Bank Hook Shot',
                            'Running Bank shot',
                            'Reverse Slam Dunk Shot',
                            'Jump Bank Hook Shot',])))

print(f'"Real" Assisted Shots')
print(json.dumps(real_shots, indent=4))

"Real" Assisted Shots
[
    "Alley Oop Dunk Shot",
    "Alley Oop Layup shot",
    "Cutting Dunk Shot",
    "Cutting Finger Roll Layup Shot",
    "Cutting Layup Shot",
    "Dunk Shot",
    "Finger Roll Layup Shot",
    "Finger Roll Shot",
    "Follow Up Dunk Shot",
    "Hook Bank Shot",
    "Hook Shot",
    "Jump Bank Hook Shot",
    "Jump Bank Shot",
    "Jump Hook Shot",
    "Jump Shot",
    "Layup Shot",
    "Reverse Dunk Shot",
    "Reverse Layup Shot",
    "Reverse Slam Dunk Shot",
    "Running Alley Oop Dunk Shot",
    "Running Alley Oop Layup Shot",
    "Running Bank Hook Shot",
    "Running Bank shot",
    "Running Dunk Shot",
    "Running Finger Roll Layup Shot",
    "Running Finger Roll Shot",
    "Running Hook Shot",
    "Running Jump Shot",
    "Running Layup Shot",
    "Running Pull-Up Jump Shot",
    "Running Reverse Dunk Shot",
    "Running Reverse Layup Shot",
    "Running Slam Dunk Shot",
    "Running Tip Shot",
    "Slam Dunk Shot"
]


In [4]:
type_map = {'Jump Shot': 1,
    'Pullup Jump shot': 79,
    'Alley Oop Dunk Shot': 52,
    'Hook Shot': 3,
    'Step Back Jump shot': 80,
    'Turnaround Jump Shot': 47,
    'Layup Shot': 5,
    'Running Pull-Up Jump Shot': 103,
    'Driving Layup Shot': 6,
    'Running Dunk Shot': 50,
    'Cutting Dunk Shot': 108,
    'Alley Oop Layup shot': 43,
    'Driving Floating Jump Shot': 101,
    'Running Jump Shot': 2,
    'Turnaround Fadeaway shot': 86,
    'Fadeaway Jump Shot': 63,
    'Floating Jump shot': 78,
    'Cutting Layup Shot': 98,
    'Running Layup Shot': 41,
    'Dunk Shot': 7,
    'Driving Dunk Shot': 9,
    'Cutting Finger Roll Layup Shot': 99,
    'Running Finger Roll Layup Shot': 76,
    'Turnaround Hook Shot': 58,
    'Running Alley Oop Dunk Shot': 106,
    'Driving Floating Bank Jump Shot': 102,
    'Running Alley Oop Layup Shot': 100,
    'Driving Reverse Layup Shot': 73,
    'Turnaround Fadeaway Bank Jump Shot': 105,
    'Driving Finger Roll Layup Shot': 75,
    'Reverse Layup Shot': 44,
    'Running Reverse Layup Shot': 74,
    'Jump Bank Shot': 66,
    'Driving Hook Shot': 57,
    'Hook Bank Shot': 67,
    'Finger Roll Layup Shot': 71,
    'Driving Bank Hook Shot': 93,
    'Reverse Dunk Shot': 51,
    'Running Reverse Dunk Shot': 110,
    'Step Back Bank Jump Shot': 104,
    'Driving Reverse Dunk Shot': 109,
    'Turnaround Bank Hook Shot': 96
}

reverse_type_map = {1: 'Jump Shot',
 79: 'Pullup Jump shot',
 52: 'Alley Oop Dunk Shot',
 3: 'Hook Shot',
 80: 'Step Back Jump shot',
 47: 'Turnaround Jump Shot',
 5: 'Layup Shot',
 103: 'Running Pull-Up Jump Shot',
 6: 'Driving Layup Shot',
 50: 'Running Dunk Shot',
 108: 'Cutting Dunk Shot',
 43: 'Alley Oop Layup shot',
 101: 'Driving Floating Jump Shot',
 2: 'Running Jump Shot',
 86: 'Turnaround Fadeaway shot',
 63: 'Fadeaway Jump Shot',
 78: 'Floating Jump shot',
 98: 'Cutting Layup Shot',
 41: 'Running Layup Shot',
 7: 'Dunk Shot',
 9: 'Driving Dunk Shot',
 99: 'Cutting Finger Roll Layup Shot',
 76: 'Running Finger Roll Layup Shot',
 58: 'Turnaround Hook Shot',
 106: 'Running Alley Oop Dunk Shot',
 102: 'Driving Floating Bank Jump Shot',
 100: 'Running Alley Oop Layup Shot',
 73: 'Driving Reverse Layup Shot',
 105: 'Turnaround Fadeaway Bank Jump Shot',
 75: 'Driving Finger Roll Layup Shot',
 44: 'Reverse Layup Shot',
 74: 'Running Reverse Layup Shot',
 66: 'Jump Bank Shot',
 57: 'Driving Hook Shot',
 67: 'Hook Bank Shot',
 71: 'Finger Roll Layup Shot',
 93: 'Driving Bank Hook Shot',
 51: 'Reverse Dunk Shot',
 110: 'Running Reverse Dunk Shot',
 104: 'Step Back Bank Jump Shot',
 109: 'Driving Reverse Dunk Shot',
 96: 'Turnaround Bank Hook Shot',
 8: 'Slam Dunk Shot',
 42: 'Driving Layup Shot',
 4: 'Tip Shot',
 65: 'Jump Hook Shot',
 46: 'Running Jump Shot',
 49: 'Driving Dunk Shot',
 56: 'Running Hook Shot',
 64: 'Follow Up Dunk Shot',
 59: 'Finger Roll Shot',
 61: 'Driving Finger Roll Shot',
 60: 'Running Finger Roll Shot',
 62: 'Turnaround Finger Roll Shot',
 54: 'Running Tip Shot',
 53: 'Tip Shot',
 45: 'Jump Shot',
 40: 'Layup Shot',
 48: 'Dunk Shot',
 55: 'Hook Shot',
 85: 'Turnaround Bank shot',
 87: 'Putback Dunk Shot',
 77: 'Driving Jump shot',
 72: 'Putback Layup Shot',
 90: 'Running Slam Dunk Shot',
 88: 'Driving Slam Dunk Shot',
 92: 'Putback Slam Dunk Shot',
 83: 'Fadeaway Bank shot',
 95: 'Running Bank Hook Shot',
 84: 'Running Bank shot',
 89: 'Reverse Slam Dunk Shot',
 82: 'Driving Bank shot',
 81: 'Pullup Bank shot',
 91: 'Putback Reverse Dunk Shot',
 94: 'Jump Bank Hook Shot',
}

In [5]:
full_df = pd.read_csv('pbp_data/2023-24.csv')

full_df = full_df.fillna({
    "HOMEDESCRIPTION": "",
    "NEUTRALDESCRIPTION": "",
    "VISITORDESCRIPTION": ""
})
full_df.loc[:, 'GAME_ID'] = [f"00{item}" for item in full_df.GAME_ID.astype(str)]

In [58]:
season = '2023-24'
shots = ShotChartDetail(team_id=0, player_id=0, context_measure_simple='FGM', season_nullable=season)

shots_df = shots.get_data_frames()[0]

,GRID_TYPE,GAME_ID,GAME_EVENT_ID,PLAYER_ID,PLAYER_NAME,TEAM_ID,TEAM_NAME,PERIOD,MINUTES_REMAINING,SECONDS_REMAINING,...,SHOT_ZONE_AREA,SHOT_ZONE_RANGE,SHOT_DISTANCE,LOC_X,LOC_Y,SHOT_ATTEMPTED_FLAG,SHOT_MADE_FLAG,GAME_DATE,HTM,VTM
0,Shot Chart Detail,0022300001,7,1626167,Myles Turner,1610612754,Indiana Pacers,1,11,41,...,Center(C),Less Than 8 ft.,2,2,21,1,1,20231103,IND,CLE
1,Shot Chart Detail,0022300001,9,1630596,Evan Mobley,1610612739,Cleveland Cavaliers,1,11,23,...,Center(C),Less Than 8 ft.,7,59,53,1,0,20231103,IND,CLE
2,Shot Chart Detail,0022300001,11,1628971,Bruce Brown,1610612754,Indiana Pacers,1,11,15,...,Center(C),Less Than 8 ft.,2,13,18,1,0,20231103,IND,CLE
3,Shot Chart Detail,0022300001,13,1631097,Bennedict Mathurin,1610612754,Indiana Pacers,1,11,9,...,Right Side Center(RC),24+ ft.,28,151,237,1,1,20231103,IND,CLE
4,Shot Chart Detail,0022300001,15,1628378,Donovan Mitchell,1610612739,Cleveland Cavaliers,1,10,56,...,Center(C),Less Than 8 ft.,2,-20,1,1,1,20231103,IND,CLE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54543,Shot Chart Detail,0022301215,605,203484,Kentavious Caldwell-Pope,1610612743,Denver Nuggets,4,0,47,...,Left Side(L),24+ ft.,22,-227,-2,1,0,20231206,LAC,DEN
54544,Shot Chart Detail,0022301215,612,1627750,Jamal Murray,1610612743,Denver Nuggets,4,0,38,...,Center(C),24+ ft.,27,-72,264,1,0,20231206,LAC,DEN
54545,Shot Chart Detail,0022301215,620,1627750,Jamal Murray,1610612743,Denver Nuggets,4,0,32,...,Center(C),Less Than 8 ft.,2,-12,26,1,1,20231206,LAC,DEN
54546,Shot Chart Detail,0022301215,637,1627750,Jamal Murray,1610612743,Denver Nuggets,4,0,23,...,Center(C),24+ ft.,28,74,271,1,1,20231206,LAC,DEN


In [17]:
league_stats = LeagueDashPlayerStats(per_mode_detailed='PerGame', season=season, season_type_all_star='Regular Season')

league_df = league_stats.get_data_frames()[0]

In [18]:
top_40 = league_df.sort_values(by='MIN', ascending=False).iloc[:40]
player_assists = full_df.loc[(full_df.PLAYER2_ID.isin(top_40.PLAYER_ID.tolist())) & (full_df.EVENTMSGTYPE == 1)]

In [19]:
player_assists.iloc[0]

GAME_ID                                                             0022301208
EVENTNUM                                                                    20
EVENTMSGTYPE                                                                 1
EVENTMSGACTIONTYPE                                                          98
PERIOD                                                                       1
WCTIMESTRING                                                           7:13 PM
PCTIMESTRING                                                             10:39
HOMEDESCRIPTION                                                               
NEUTRALDESCRIPTION                                                            
VISITORDESCRIPTION           Embiid 1' Cutting Layup Shot (6 PTS) (Harris 1...
SCORE                                                                    6 - 2
SCOREMARGIN                                                                 -4
PERSON1TYPE                                         

In [20]:
top_40

,PLAYER_ID,PLAYER_NAME,NICKNAME,TEAM_ID,TEAM_ABBREVIATION,AGE,GP,W,L,W_PCT,...,BLK_RANK,BLKA_RANK,PF_RANK,PFD_RANK,PTS_RANK,PLUS_MINUS_RANK,NBA_FANTASY_PTS_RANK,DD2_RANK,TD3_RANK,WNBA_FANTASY_PTS_RANK
476,1630178,Tyrese Maxey,Tyrese,1610612755,PHI,23.0,19,13,6,0.684,...,103,12,182,35,13,15,16,33,11,17
156,1627832,Fred VanVleet,Fred,1610612745,HOU,29.0,17,9,8,0.529,...,145,87,216,86,75,61,46,21,11,49
435,1631101,Shaedon Sharpe,Shaedon,1610612757,POR,20.0,20,6,14,0.300,...,166,21,33,35,60,464,76,91,11,65
356,1628970,Miles Bridges,Miles,1610612766,CHA,25.0,9,3,6,0.333,...,200,224,121,55,49,456,52,50,11,47
291,201142,Kevin Durant,Kevin,1610612756,PHX,35.0,19,10,9,0.526,...,31,108,171,4,3,48,8,40,11,7
111,1628368,De'Aaron Fox,De'Aaron,1610612758,SAC,25.0,14,9,5,0.643,...,192,32,42,8,4,86,9,64,11,8
233,1628369,Jayson Tatum,Jayson,1610612738,BOS,25.0,20,15,5,0.750,...,194,54,126,33,10,10,15,21,11,10
452,1626179,Terry Rozier,Terry,1610612766,CHA,29.0,10,3,7,0.300,...,133,69,217,47,31,433,31,91,11,35
462,1629027,Trae Young,Trae,1610612737,ATL,25.0,19,8,11,0.421,...,346,26,258,3,12,233,11,8,11,9
403,202331,Paul George,Paul,1610612746,LAC,33.0,20,10,10,0.500,...,276,100,37,43,25,24,30,91,11,25


In [25]:
player_score = player_assists.loc[player_assists.PLAYER1_ID.isin(league_df.PLAYER_ID.tolist())]

assists_players = player_score.groupby(['PLAYER1_ID', 'PLAYER2_ID']).size().reset_index()



In [28]:
assists_players = assists_players.rename(columns={0: 'COUNT'})

assists_players.sort_values(by=['COUNT'], ascending=False)

,PLAYER1_ID,PLAYER2_ID,COUNT
339,1630578,1627832,60
92,203954,1630178,51
112,1626167,1630169,50
218,1629008,203999,46
68,203484,203999,42
...,...,...,...
102,204456,1630169,1
64,203458,1628368,1
372,1631121,1626179,1
230,1629052,1628369,1


In [29]:
name_only = league_df.loc[:, ['PLAYER_ID', 'PLAYER_NAME']]

In [31]:
merged_df = pd.merge(left=assists_players, right=name_only, left_on=['PLAYER1_ID'], right_on='PLAYER_ID')

In [33]:
del merged_df['PLAYER_ID']

merged_df = merged_df.rename(columns={'PLAYER_NAME': 'SCORER_NAME'})

In [35]:
merged_df = pd.merge(left=merged_df, right=name_only, left_on=['PLAYER2_ID'], right_on='PLAYER_ID')

del merged_df['PLAYER_ID']

merged_df = merged_df.rename(columns={'PLAYER_NAME': 'ASSISTER_NAME'})

In [49]:
top_combinations = merged_df.sort_values(by='COUNT', ascending=False)

top_combinations[top_combinations.PLAYER1_ID == 202695]
top_combinations.tail(20)

,PLAYER1_ID,PLAYER2_ID,COUNT,SCORER_NAME,ASSISTER_NAME
164,1627788,203954,1,Furkan Korkmaz,Joel Embiid
24,203496,202695,1,Robert Covington,Kawhi Leonard
31,200782,202699,1,P.J. Tucker,Tobias Harris
358,1631200,1631101,1,Kris Murray,Shaedon Sharpe
356,1630703,1631101,1,Scoot Henderson,Shaedon Sharpe
147,1641748,203507,1,Andre Jackson Jr.,Giannis Antetokounmpo
150,203458,1628368,1,Alex Len,De'Aaron Fox
337,1641711,1630567,1,Gradey Dick,Scottie Barnes
334,1629667,1630567,1,Jalen McDaniels,Scottie Barnes
30,1631217,202695,1,Moussa Diabate,Kawhi Leonard


In [56]:
player_assists = full_df.loc[(full_df.PLAYER2_ID != 0) & (full_df.EVENTMSGTYPE == 1)]
assists_combo = player_assists.groupby(['PLAYER1_ID', 'PLAYER2_ID', 'PLAYER1_NAME', 'PLAYER2_NAME']).size().reset_index()
assists_combo = assists_combo.rename(columns={0: 'COUNT'})
assists_combo = assists_combo.sort_values(by='COUNT', ascending=False)

assists_combo[assists_combo.PLAYER2_ID == 1630169]

# assists_combo

,PLAYER1_ID,PLAYER2_ID,PLAYER1_NAME,PLAYER2_NAME,COUNT
811,1626167,1630169,Myles Turner,Tyrese Haliburton,50
2073,1630167,1630169,Obi Toppin,Tyrese Haliburton,38
949,1627741,1630169,Buddy Hield,Tyrese Haliburton,30
1394,1628971,1630169,Bruce Brown,Tyrese Haliburton,21
2138,1630174,1630169,Aaron Nesmith,Tyrese Haliburton,19
2859,1631097,1630169,Bennedict Mathurin,Tyrese Haliburton,17
2205,1630188,1630169,Jalen Smith,Tyrese Haliburton,17
1734,1629614,1630169,Andrew Nembhard,Tyrese Haliburton,5
2524,1630543,1630169,Isaiah Jackson,Tyrese Haliburton,2
3364,1641767,1630169,Ben Sheppard,Tyrese Haliburton,1


In [57]:
full_df[full_df.EVENTMSGTYPE == 1]

,GAME_ID,EVENTNUM,EVENTMSGTYPE,EVENTMSGACTIONTYPE,PERIOD,WCTIMESTRING,PCTIMESTRING,HOMEDESCRIPTION,NEUTRALDESCRIPTION,VISITORDESCRIPTION,...,PLAYER2_TEAM_ABBREVIATION,PERSON3TYPE,PLAYER3_ID,PLAYER3_NAME,PLAYER3_TEAM_ID,PLAYER3_TEAM_CITY,PLAYER3_TEAM_NICKNAME,PLAYER3_TEAM_ABBREVIATION,VIDEO_AVAILABLE_FLAG,PCTIMESECONDS
2,0022301208,7,1,5,1,7:12 PM,11:47,,,Embiid 2' Layup (2 PTS) (Batum 1 AST),...,PHI,0,0,NaN,NaN,NaN,NaN,NaN,1,13
5,0022301208,12,1,73,1,7:12 PM,11:08,,,Embiid 3' Driving Reverse Layup (4 PTS),...,NaN,0,0,NaN,NaN,NaN,NaN,NaN,1,52
8,0022301208,15,1,87,1,7:13 PM,10:58,Gafford Putback Dunk (2 PTS),,,...,NaN,0,0,NaN,NaN,NaN,NaN,NaN,1,62
11,0022301208,20,1,98,1,7:13 PM,10:39,,,Embiid 1' Cutting Layup Shot (6 PTS) (Harris 1...,...,PHI,0,0,NaN,NaN,NaN,NaN,NaN,1,81
16,0022301208,26,1,1,1,7:14 PM,10:08,Gafford 14' Jump Shot (4 PTS) (Avdija 1 AST),,,...,WAS,0,0,NaN,NaN,NaN,NaN,NaN,1,112
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
144366,0022300062,695,1,79,4,12:34 AM,2:57,Paul 18' Pullup Jump Shot (14 PTS),,,...,NaN,0,0,NaN,NaN,NaN,NaN,NaN,1,543
144381,0022300062,715,1,1,4,12:38 AM,1:09,,,Okogie 3PT Jump Shot (17 PTS) (Booker 6 AST),...,PHX,0,0,NaN,NaN,NaN,NaN,NaN,1,651
144385,0022300062,721,1,1,4,12:40 AM,0:44,,,Gordon 29' 3PT Jump Shot (10 PTS) (Booker 7 AST),...,PHX,0,0,NaN,NaN,NaN,NaN,NaN,1,676
144388,0022300062,725,1,80,4,12:40 AM,0:31,Curry 24' 3PT Step Back Jump Shot (27 PTS) (Ku...,,,...,GSW,0,0,NaN,NaN,NaN,NaN,NaN,1,689
